```bash
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh

chmod +x Miniconda3-latest-Linux-x86_64.sh

./Miniconda3-latest-Linux-x86_64.sh

Please, press ENTER to continue
```

Einmal Leertaste drücken.

```bash
Please answer 'yes' or 'no':'
>>> yes

Press ENTER to confirm the location

Do you wish the installer to initialize Miniconda3
by running conda init? [yes|no]
[no] >>> yes

conda config --set auto_activate_base false

conda create -n python_workshop_env python=3

Proceed ([y]/n)? y

. ~/.bashrc

conda activate python_workshop_env

conda install s3fs bokeh dask ipykernel numpy pandas "dask[distributed]" mpi4py

Proceed ([y]/n)? y

conda install -c conda-forge dask-mpi
```


In [3]:
#import os

#f = open(os.path.expanduser("~/job_dask_mpi.sh"), "w")
#f.write("""#!/bin/bash -l
#conda activate python_workshop_env
#mpirun -np 20 dask-mpi --scheduler-file ~/dask-scheduler.json --interface='ib0' --local-directory='/tmp'""")
#f.close()

```bash
tar -zcvf ~/miniconda3/envs/python_workshop_env.gz ~/miniconda3/envs/python_workshop_env/
```

In [49]:
import os

f = open(os.path.expanduser("~/job_dask_mpi.sh"), "w")
f.write("""#!/bin/bash -l
if [ ! -d "/tmp/envs" ]; then
mkdir /tmp/envs
cp -r --preserve=links ~/miniconda3/envs/python_workshop_env.gz /tmp/envs
tar -zxvf /tmp/envs/python_workshop_env.gz --directory /tmp/envs
fi

conda activate /tmp/envs/python_workshop_env
mpirun -np 9 dask-mpi --scheduler-file ~/dask-scheduler.json --interface='ib0' --local-directory='/tmp' --no-nanny""")
f.close()


```bash
rm -f ~/dask-scheduler.json  && sbatch -p dev_multiple --nodes=2 --ntasks=9 --ntasks-per-node=5 --time=30:00 --mem=90000mb ./job_dask_mpi.sh
```

In [50]:
import os

#os.system("rm -f ~/dask-scheduler.json  && sbatch -p dev_multiple --nodes=2 --ntasks=9 --ntasks-per-node=5 --time=30:00 --mem=90000mb ~/job_dask_mpi.sh")
#TODO: warum bricht der Job über os.system immer nach <1min. ab?

Submitted batch job 20135577


0

In [51]:
import os, time

while not os.path.isfile(os.path.expanduser("~/dask-scheduler.json")):
    time.sleep(1)

KeyboardInterrupt: 

In [43]:
from dask.distributed import Client
client = Client(scheduler_file=os.path.expanduser('~/dask-scheduler.json'))

In [44]:
#from dask.distributed import Client
#client = Client('172.26.20.1:40687')

In [46]:
client

<Client: 'tcp://172.26.20.1:39845' processes=20 threads=1600, memory=1.72 TiB>

In [47]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import datetime
import s3fs

taxi = dd.read_csv(
    's3://nyc-tlc/trip data/yellow_tripdata_2019-*.csv',
    assume_missing=True, #Beim Einlesen werden alle Ints zu Floats. Dies erlaubt fehlende Werte
    parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'], #Interpretiert diese Spalten als Datum
    storage_options={'anon': True}, #Für S3: KeineAuthentifizierung für diesen Bucket nötig
)

print(f"Row count: {len(taxi)}") #Zeilen aller Datensätze zusammen
print(f"Size in GB: {taxi.memory_usage(deep=True).sum().compute() / 1e9}")

Row count: 84399019
Size in GB: 16.367014316
